In [1]:
import pandas as pd
import numpy as np
import numpy.typing as npt
from torchvision.datasets import Caltech101, Caltech256

# Get label names
caltech101_labels = Caltech101(root="datasets/caltech101", download=False).categories
caltech256_labels = Caltech256(root="datasets/caltech256", download=False).categories

# Get targets
caltech101_targets = pd.read_csv("outputs/caltech101.csv")
caltech256_targets = pd.read_csv("outputs/caltech256.csv")
caltech101_model_caltech256 = pd.read_csv("outputs/caltech101_model_caltech256.csv")
caltech256_model_caltech101 = pd.read_csv("outputs/caltech256_model_caltech101.csv")

In [2]:
from taxonomy import Taxonomy

taxonomy = Taxonomy(
    a_to_b_predictions=caltech101_model_caltech256["predictions"].to_numpy(),
    a_targets=caltech101_targets["targets"].to_numpy(),
    b_to_a_predictions=caltech256_model_caltech101["predictions"].to_numpy(),
    b_targets=caltech256_targets["targets"].to_numpy(),
)

val = taxonomy.build_universal_taxonomy()
rels = val.get_relationships()

print(rels)

{((1, 245), UniversalClass({(1, np.int64(65)), (0, 39)}), np.float32(0.16666667)), ((1, 182), UniversalClass({(0, np.int64(6))}), np.float32(0.3)), ((0, 14), UniversalClass({(1, 63), (0, np.int64(36))}), np.float32(0.5)), ((0, 20), UniversalClass({(0, np.int64(23)), (1, 35)}), np.float32(0.25)), ((1, 213), UniversalClass({(0, np.int64(55)), (1, 122)}), np.float32(0.625)), ((1, 210), UniversalClass({(0, np.int64(80))}), np.float32(0.5)), ((1, 198), UniversalClass({(0, np.int64(80))}), np.float32(0.5)), ((0, np.int64(43)), UniversalClass({(0, np.int64(43))}), 1.0), ((0, np.int64(84)), UniversalClass({(1, 192), (0, np.int64(84))}), 1.0), ((1, 15), UniversalClass({(1, 237), (0, np.int64(17))}), np.float32(0.8)), ((1, 63), UniversalClass({(1, 63), (0, np.int64(36))}), 1.0), ((0, 95), UniversalClass({(1, np.int64(72)), (0, 95)}), 1.0), ((0, np.int64(59)), UniversalClass({(0, np.int64(59))}), 1.0), ((1, 48), UniversalClass({(1, 113), (0, np.int64(51))}), np.float32(0.3)), ((1, np.int64(149)),